In [1]:
from numba import cuda
cuda.select_device(0)
cuda.close()
print('CUDA memory released: GPU0')

CUDA memory released: GPU0


In [2]:
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

from models.unets import Unet2D
from models.deeplab import Deeplabv3, relu6, DepthwiseConv2D, BilinearUpsampling
#from models.separable_unet import Separable_Unet2D
from models.FCN import FCN_Vgg16_16s
from models.SegNet import SegNet

from utils.learning.metrics import dice_coef, precision, recall
from utils.learning.losses import dice_coef_loss
from utils.io.data import DataGen, save_results, save_history, load_data

# Check the system information

In [4]:
#######################################################################################################
#check the system information, check if cuda and gpu computing for tensorflow is installed properly
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib

print("whether tensorflow is built with cuda: ", tf.test.is_built_with_cuda())
print("whether gpu computing is available for tensorflow: ", tf.test.is_gpu_available())
#print("whether gpu computing is available for tensorflow: ",tf.config.list_physical_devices('GPU'))
print("using keras version: ", keras.__version__)
print("using tensorflow version: ", tf.__version__)
print("\n")
print("Device details:\n", device_lib.list_local_devices())
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

whether tensorflow is built with cuda:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
whether gpu computing is available for tensorflow:  False
using keras version:  2.9.0
using tensorflow version:  2.9.1


Device details:
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14995399277592272260
xla_global_id: -1
]
Please install GPU version of TF


2022-08-25 12:49:43.279594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-25 12:49:43.280314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 12:49:43.288079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aravind/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64
2022-08-25 12:49:43.288094: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some

In [4]:
# For TF 1.0, set the GPU memory growth
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.8
#config.gpu_options.allow_growth = True
#sess = tf.InteractiveSession(config=config)

# For TF 2.0, set the GPU memory growth
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#from tensorflow import ConfigProto
#from tensorflow import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2022-08-25 12:45:02.224762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 12:45:02.224906: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# Varibales and data generator
input_dim_x=224
input_dim_y=224
n_filters = 32
dataset = 'benchmark_datasets/azh_wound_dataset'#'merged_azh_dog1_8_2800'#'dog1_8_1400'#'dog1_8'#'azh_wound_care_center_dataset_patches'#'Medetec_foot_ulcer_224'#'merged_azh_dog1_8'
data_gen = DataGen('./data/' + dataset + '/', split_ratio=0.2, x=input_dim_x, y=input_dim_y)

In [ ]:
######### Get the deep learning models #########

######### Unet ##########
#unet2d = Unet2D(n_filters=n_filters, input_dim_x=None, input_dim_y=None, num_channels=3)
# unet-1
#model, model_name = unet2d.get_unet_model_yuanqing()
# unet-2
#model, model_name = unet2d.get_unet_2_model()

######### Seperable Unet ##########
# Not working now, has package import issue
# sep_unet = Seperable_Unet2D(n_filters=n_filters, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
# model, model_name = sep_unet.get_seperable_unet_model_5_levels()
# model, model_name = sep_unet.get_sep_unet_v3()

######### MobilenetV2 and DeepLabV3+#####################
# Not working now, has package import issue
#model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
#model_name = 'DeepLabV3+'
#with CustomObjectScope({'relu6': relu6,'DepthwiseConv2D': DepthwiseConv2D, 'BilinearUpsampling': BilinearUpsampling}):
#    model = load_model('azh_wound_care_center_diabetic_foot_training_history/2020-02-10 02:57:27.555495.hdf5'
#                       , custom_objects={'dice_coef': dice_coef, 'precision':precision, 'recall':recall})

######### Vgg16 ##########
#model, model_name = FCN_Vgg16_16s(input_shape=(input_dim_x, input_dim_y, 3))

######### SegNet ##########
segnet = SegNet(n_filters, input_dim_x, input_dim_y, num_channels=3)
# SegNet-1
#model, model_name = segnet.get_SegNet()
# SegNet-2
#model, model_name = segnet.get_SegNet_2()

# SegNet-3
model, model_name = segnet.get_SegNet_3()

#plot_model(model, to_file=model_name+'.png')

In [ ]:
# training
batch_size = 2
epochs = 2000
learning_rate = 1e-4
loss = 'binary_crossentropy'

# restore_best_weights need at least tf version 1.15 and keras version 2.2.3
# currently using tf 1.10 and keras 2.1.3
#es = EarlyStopping(monitor='val_dice_coef', patience=200, mode='max', restore_best_weights=True)
es = EarlyStopping(monitor='val_dice_coef', patience=200, mode='max')
#training_history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs
#                             , validation_split=0.2, verbose=1, callbacks=[])

model.summary()
model.compile(optimizer=Adam(lr=learning_rate), loss=loss, metrics=[dice_coef, precision, recall])
# model.fit_generator
training_history = model.fit_generator(data_gen.generate_data(batch_size=batch_size, train=True),
                                       steps_per_epoch=int(data_gen.get_num_data_points(train=True) / batch_size),
                                       callbacks=[es],
                                       validation_data=data_gen.generate_data(batch_size=batch_size, val=True),
                                       validation_steps=int(data_gen.get_num_data_points(val=True) / batch_size),
                                       epochs=epochs,
                                       max_queue_size = 1,
                                       use_multiprocessing=False)

In [ ]:
### save the model weight file and its training history
save_history(model, model_name, training_history, dataset, n_filters, epochs, learning_rate, loss, color_space='RGB',
             path='./trained_models/')

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()
print('CUDA memory released: GPU0')